In [137]:
import tensorflow as tf
import numpy as np
import csv
import pandas as pd
from sklearn.cross_validation import KFold

if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
    

DATA      = r'C:\Users\Ekpyrotic\Documents\Tensorflow stuff\binding affinity predictor\one_hot_encoded_dat.csv'
HLA_distances  = r'C:\Users\Ekpyrotic\Documents\Tensorflow stuff\binding affinity predictor\crystal structures\HLAdistmatrix.csv'
HLA_dist = np.genfromtxt(HLA_distances, delimiter=',')
#1: HLA-A*:0201
#2: HLA-B*58:01
#3: HLA-B*57:01

newrows   = []
with open(DATA) as csvfile:
    i = 0
    filereader = csv.reader(csvfile)
    for row in filereader:
        newrow = row
        if row[0] == 'HLA-A*02:01':
            i = i + 1
            newrow[181] = HLA_dist[0,0]
        elif row[0] == 'HLA-B*57:01':
            newrow[181] = HLA_dist[1,0]
        elif row[0] == 'HLA-B*57:01':
            newrow[181] = HLA_dist[2,0]
        else:
            newrow = []         #get rid of other HLA types
            #newrow[181] = 999   #keep other HLA type rows
        if newrow:
            newrows.append(newrow)

j = int(np.floor(i/5))

#column 0: HLA type
#column 1-181: one hot encoding of aa values
#column 182: HLA PH distance calc
#column 183: observed binding affinity

arrayform = np.asarray(newrows)
train     = arrayform[j:,1:].astype(np.float)
train_x   =  train[:, 0:180]
train_y   =  train[:,181]

test      = arrayform[0:j,1:].astype(np.float)
test_set_data   =  test[:, 0:180]
test_set_target =  test[:,181]


np.random.shuffle(cleandat)
folds = KFold(train_x.shape[0], n_folds= 10)
foldvals = list(folds)


def get_train_inputs(i):
    train, test = foldvals[i]
    x = tf.constant(train_x[train,:-1])
    y = tf.constant(train_y[train])
    #y = tf.transpose(y)
    return x, y

def get_test_inputs():
    x = tf.constant(test_set_data[:,:-1])
    y = tf.constant(test_set_target)
    return x, y



In [ ]:
#train, test = foldvals[0]
#print(train)
print(train.shape)
print(train_x.shape[0])

In [ ]:
#MODEL ONE: NO TOPOLOGY 


features = [tf.contrib.layers.real_valued_column("",dimension = 180)]       #9mer data + distance from HLA a0201 
print(get_train_inputs())
print(features)

classifier = tf.contrib.learn.DNNRegressor(hidden_units = [20,10,10],feature_columns=features,
                                            optimizer=tf.train.AdamOptimizer,
                                            #l2_regularization_strength=0.001,
                                            #model_dir='/tmp/mhc_model'
                                            )


classifier.fit(input_fn = get_train_inputs, steps = 2000)
#classifier.fit(x=training_set_data, y=training_set_target, steps=20)
#classifier.fit(x=training_set_data, y=training_set_target, steps=1000)



In [ ]:
#MODEL ONE: NO TOPOLOGY

accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                     steps=100)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


In [138]:
#MODEL TWO: TOPOLOGY

#features = [tf.contrib.layers.real_valued_column("",dimension = 181)]       #9mer data + distance from HLA a0201 
features = [tf.contrib.layers.real_valued_column("",dimension = 180)] 
#print(get_train_inputs())
#print(features)
accuracy_score = np.zeros(10)  
tf.logging.set_verbosity(tf.logging.ERROR)

for i in range(1,10):
    classifier = tf.contrib.learn.DNNRegressor(hidden_units = [20,10,10],feature_columns=features,
                                            optimizer=tf.train.ProximalAdagradOptimizer(
                                            learning_rate=0.01,
                                            l1_regularization_strength=0.001),
                                            )

    classifier.fit(input_fn = lambda:get_train_inputs(i), steps = 200)
    accuracy_score[i] = classifier.evaluate(input_fn=get_test_inputs,steps = 50)['loss']
    print(i)

#classifier.fit(x=training_set_data, y=training_set_target, steps=20)
#classifier.fit(x=training_set_data, y=training_set_target, steps=1000)
#evaluate


print("\nTest MSE: {0:f}\n".format(np.average(accuracy_score)))



1
2
3
4
5
6
7
8
9

Test MSE: 3108181632.000000



In [130]:
classifier.evaluate(input_fn=get_test_inputs,steps = 20)


{'global_step': 20, 'loss': 3.4737761e+09}